# Open Router testing

https://www.openrouter.ai

### What is possible?
With open router you can use the same interface to interact with hundreds of models, while the interaction style & api key stays the same.

We can use openAI SDk to interace with openAI, but can we also for all the other models?

### 
In this jupyter notebook. I am making interesting plots using usage data from openrouter

In [ ]:
# Import libraries
import requests
import os
import dotenv
dotenv.load_dotenv()
import json

In [ ]:

url = "https://openrouter.ai/api/v1/models"

headers = {"Authorization": f"Bearer {os.getenv("OPEN_ROUTER_API_KEY")}"}

response = requests.get(url, headers=headers).json()["data"]

print(json.dumps(response, indent=4))


### Reverse engineer openrouter backend

They do not share total usage statistics of their models sadly. But we can infer it anyway by reverse engineering their frontend API

The route `https://openrouter.ai/api/frontend/models/find` allows us to get information about usage statistics from the front end api. It can have a query parameter: such as `https://openrouter.ai/api/frontend/models/find?order=top-weekly`

The query paramter can have the possible values of :
`latency-low-to-high`
`throughput-high-to-low`
`context-high-to-low`
`pricing-high-to-low`
`pricing-low-to-high`
`top-weekly`
`newest`

### What the api returns

The result of the API is a dict with three parent keys:
- `categories` -> 0.5% of prompts are sampled, the purpose is determined for the sample. Each prompt is analyzed for what the purpose is. Which contributes to the category
- `analytics` -> 366 different models, free variant is counted separately, also other variants are counted separately
- `model` -> List of models Sorted order or models based on the query param

In [ ]:
from typing import Literal, Optional
def get_models_standard_dev_api():
    
    url = "https://openrouter.ai/api/v1/models"

    response = requests.get(url).json()["data"]

    return response

def get_analytics_openrouter(order_param: Optional[Literal[
                                                          "latency-low-to-high",
                                                          "throughput-high-to-low",
                                                          "context-high-to-low",
                                                          "pricing-high-to-low",
                                                          "pricing-low-to-high",
                                                          "top-weekly",
                                                          "newest"  
                                                          ]] = None):
    """ returns the frontend json of the usage statistics of the models on openrouter
    
    returns dict with values ['models', 'analytics', 'categories']
    
    the analytics dictionary is what is super useful data about usage
    
    categories returns the top 10 models for that usage category. Such as legal, classification, 
    based on usage from other openrouter users
    
    models is similar to what you see from the get_all_models route from the official api"""
    if order_param is not None:
        response = requests.get(f"https://openrouter.ai/api/frontend/models/find?order={order_param}")
    else:
        response = requests.get(f"https://openrouter.ai/api/frontend/models/find")

    json_response = response.json()["data"]
    return json_response

find_order_by_list = [
  "latency-low-to-high",
"throughput-high-to-low",
"context-high-to-low",
"pricing-high-to-low",
"pricing-low-to-high",
"top-weekly",
"newest"  
]
get_models_standard_dev_api()

In [ ]:
import datetime
# we log some of the date to see if the response of openrouter changes frequently or not. Or does it just use the same data each
with open(f"openrouter_data/openrouter_frontend_{datetime.datetime.now().strftime("%d-%m-%Y")}.json", "w") as file:
    file.write(json.dumps(requests.get(f"https://openrouter.ai/api/frontend/models/find?view=month").json()))

In [415]:
requests.get(f"https://openrouter.ai/api/frontend/models/find").json()

{'data': {'models': [{'slug': 'anthropic/claude-opus-4.5',
    'hf_slug': '',
    'updated_at': '2025-11-24T19:03:09.404215+00:00',
    'created_at': '2025-11-24T18:56:20+00:00',
    'hf_updated_at': None,
    'name': 'Anthropic: Claude Opus 4.5',
    'short_name': 'Claude Opus 4.5',
    'author': 'anthropic',
    'description': 'Claude Opus 4.5 is Anthropic’s frontier reasoning model optimized for complex software engineering, agentic workflows, and long-horizon computer use. It offers strong multimodal capabilities, competitive performance across real-world coding and reasoning benchmarks, and improved robustness to prompt injection. The model is designed to operate efficiently across varied effort levels, enabling developers to trade off speed, depth, and token usage depending on task requirements. It comes with a new parameter to control token efficiency, which can be accessed using the OpenRouter Verbosity parameter with low, medium, or high.\n\nOpus 4.5 supports advanced tool use

In [ ]:
import datetime 



### The order variable has no effect on the number of models that are ranked

In [ ]:
print(len(get_analytics_openrouter()["analytics"]))
print(len(get_analytics_openrouter(find_order_by_list[0])["analytics"]))
print(len(get_analytics_openrouter(find_order_by_list[1])["analytics"]))


### The regular frontend api & backend api are not the same, even though they should give you the same information. IT is named differently

In [ ]:
print(sorted(get_models_standard_dev_api()[0].keys()))
print(sorted(get_analytics_openrouter()["models"][0].keys()))


In [ ]:
model_analytics = get_analytics_openrouter()["analytics"]

In [ ]:
free_models = [model for model in sorted(model_analytics.keys()) if "free" in model]
print(f"there are {len(free_models)} models tha we can use for free \n {free_models}")

### Each model that there is an analysis object for. Is represeted by the key and values as below:

```python{'date': '2025-11-14 00:00:00',
 'model_permaslug': 'alibaba/tongyi-deepresearch-30b-a3b',
 'variant': 'free',
 'variant_permaslug': 'alibaba/tongyi-deepresearch-30b-a3b:free',
 'count': 125619,
 'total_completion_tokens': 205142552,
 'total_prompt_tokens': 687785342,
 'total_native_tokens_reasoning': 158213434,
 'num_media_prompt': 0,
 'num_media_completion': 0,
 'num_audio_prompt': 0,
 'total_native_tokens_cached': 307326294,
 'total_tool_calls': 12129,
 'requests_with_tool_call_errors': 1966}```


In [ ]:
model_analytics['alibaba/tongyi-deepresearch-30b-a3b:free']

In [ ]:
get_models_standard_dev_api()

In [ ]:
get_analytics_openrouter()

# Building the new data Model

In [ ]:
from typing import List
from pydantic import BaseModel
from collections import defaultdict
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np


class ModelPricing(BaseModel):
    """ information from each of the model relating to its pricing cost"""
    prompt: float
    completion: float
    request: Optional[float] = 0
    image: Optional[float] = 0
    web_search: Optional[float] = 0
    internal_reasoning: Optional[float] = 0


class ModelData(BaseModel):
    model_id: str
    total_tool_calls: int
    tool_call_errors: int
    total_completion_tokens: int
    total_prompt_tokens: int
    total_native_reasoning_tokens: int
    pricing: ModelPricing
    times_used: int
    free_available: bool = False
    
    def total_tokens_used(self):
        return self.total_completion_tokens + self.total_native_reasoning_tokens + self.total_prompt_tokens
    
    def total_completion_spend(self):
        return self.total_completion_tokens * self.pricing.completion
    
    def total_reasoning_spend(self):
        return self.total_native_reasoning_tokens * self.pricing.internal_reasoning
    
    def total_prompt_spend(self):
        return self.total_prompt_tokens * self.pricing.prompt
    
    def total_spend(self):
        return self.total_completion_spend() * self.total_reasoning_spend() + self.total_prompt_spend()
    
    def error_rate(self):
        if self.total_tool_calls == 0:
            return 0
        return self.tool_call_errors/self.total_tool_calls
    
    def model_provider(self):
        return self.model_id.split("/")[0]

    def model_name(self):
        return self.model_id.split("/")[-1]


class OpenRouterModelData(BaseModel):
    model_data: List[ModelData] = []

    @staticmethod
    def get_model_data_with_analytics():
        """
        {'id': 'allenai/olmo-3-32b-think',
        'canonical_slug': 'allenai/olmo-3-32b-think-20251121',
        'hugging_face_id': 'allenai/Olmo-3-32B-Think',
        'name': 'AllenAI: Olmo 3 32B Think',
        'created': 1763758276,
        'description': 'Olmo 3 32B Think is a large-scale, 32-billion-parameter model purpose-built for deep reasoning, complex logic chains and advanced instruction-following scenarios. Its capacity enables strong performance on demanding evaluation tasks and highly nuanced conversational reasoning. Developed by Ai2 under the Apache 2.0 license, Olmo 3 32B Think embodies the Olmo initiative’s commitment to openness, offering full transparency across weights, code and training methodology.',
        'context_length': 65536,
        'architecture': {'modality': 'text->text',
        'input_modalities': ['text'],
        'output_modalities': ['text'],
        'tokenizer': 'Other',
        'instruct_type': None},
        'pricing': {'prompt': '0.0000002',
        'completion': '0.00000035',
        'request': '0',
        'image': '0',
        'web_search': '0',
        'internal_reasoning': '0'},
        'top_provider': {'context_length': 65536,
        'max_completion_tokens': 65536,
        'is_moderated': False},
        'per_request_limits': None,
        'supported_parameters': ['frequency_penalty',
        'include_reasoning',
        'logit_bias',
        'max_tokens',
        'min_p',
        'presence_penalty',
        'reasoning',
        'repetition_penalty',
        'response_format',
        'seed',
        'stop',
        'structured_outputs',
        'temperature',
        'top_k',
        'top_p'],
        'default_parameters': {'temperature': 0.6,
        'top_p': 0.95,
        'frequency_penalty': None},
        'analytics': {'date': '2025-11-21 00:00:00',
        'model_permaslug': 'allenai/olmo-3-32b-think-20251121',
        'variant': 'standard',
        'variant_permaslug': 'allenai/olmo-3-32b-think-20251121',
        'count': 26850,
        'total_completion_tokens': 59912743,
        'total_prompt_tokens': 38135902,
        'total_native_tokens_reasoning': 54764459,
        'num_media_prompt': 0,
        'num_media_completion': 0,
        'num_audio_prompt': 0,
        'total_native_tokens_cached': 22299584,
        'total_tool_calls': 0,
        'requests_with_tool_call_errors': 0}}
        """
        standard_api_response = get_models_standard_dev_api()
        model_analytics = get_analytics_openrouter()["analytics"]
        models_analytics_improved = []
        slug_counter = defaultdict(int)
        for index, model in enumerate(standard_api_response):
            slug = model["canonical_slug"]
            
            
            # We try here to link the frontend api to the backend api data format of the models. 
            # There are some issues, so we fixed those with trial and error as seen below
            # Janky but works 
            try:
                
                analytics = model_analytics[slug]
            except KeyError:
                if "google/gemini-2.5-pro" in  slug:
                    slug = "google/gemini-2.5-pro"
                elif "beta" in slug:
                    slug = slug.replace("-beta", "")
                # elif "x-ai/grok-3-mini" in slug:
                #     slug = "x-ai/grok-3-mini"
                # elif "x-ai/grok-3" in slug:
                #     slug = "x-ai/grok-3"
                
                elif "openrouter/auto" in slug:
                    continue
                else:
                    slug += ":free"
                    
                analytics = model_analytics[slug]
            
            finally:
                updated_model = model.copy()
                updated_model["analytics"] = analytics
                models_analytics_improved.append(updated_model)
                slug_counter[slug] +=1
                if slug_counter[slug] >1:
                    print(updated_model["id"])
        
        return models_analytics_improved
          
    @classmethod
    def from_api_data(cls, minimum_tool_calls: int = 1000, only_free_models: bool = None):
      """Create OpenRouterModelData from API responses"""      
      models_analytics_improved = OpenRouterModelData.get_model_data_with_analytics()
      model_data_error_rate: List[ModelData] = []
      print(len(models_analytics_improved))
      
      for model_data in models_analytics_improved:
        model = model_data["analytics"]
        model_has_free =  "free" in model_data["id"]
        # When we want to skip certain models skip the ones that are not part of the subset of interest 
        if (only_free_models is not None and model_has_free != only_free_models):
            continue
            

        if model.get("total_tool_calls") >= minimum_tool_calls:

            single_model_data = ModelData(
                model_id=model_data["id"],
                total_tool_calls=model["total_tool_calls"],  # total tool calls 
                total_completion_tokens=model["total_completion_tokens"], # completion tokens
                total_prompt_tokens=model["total_prompt_tokens"],  # prompt tokens
                total_native_reasoning_tokens=model["total_native_tokens_reasoning"], # reason tokens
                times_used=model["count"], # how often model been used
                pricing=model_data["pricing"], # Completion token price
                tool_call_errors=model["requests_with_tool_call_errors"],
                free_available= model_has_free
                )
            model_data_error_rate.append(single_model_data)
        else:
            print(model_data["id"])

        
      sorted_data = sorted(model_data_error_rate, key=lambda x: x.model_provider())
      return cls(model_data=sorted_data)
      
    def model_error_rates(self):
        return [model.error_rate() for model in self.model_data]
    
    def model_completion_prices(self, in_MTOK: bool = True):
        """MNOK = when to multiply the token cost times 1 million. To get $ per MTOK"""
        multiplyer = 1000000 if in_MTOK else 1
        
        return [model.pricing.completion*multiplyer for model in self.model_data]
    
    def model_total_tokens(self):
        return [model.total_tokens_used() for model in self.model_data]
    
    def model_model_names(self):
        return [model.model_name() for model in self.model_data]
    
    def model_providers(self, unique_set: bool = False):
        providers = [model.model_provider() for model in self.model_data]
        return set(providers) if unique_set else providers
    
    def model_tool_calls(self):
        return [model.total_tool_calls for model in self.model_data]
    
    def model_total_spend(self):
        return [model.total_spend() for model in self.model_data]
    
    def model_free_available(self):
        return [model.free_available for model in self.model_data]
    
    def total_count_used(self):
        return [model.times_used for model in self.model_data]
    
    def tokens_per_request(self):
        total_tokens = np.array(self.model_total_tokens())
        total_calls = np.array(self.total_count_used())
        return total_tokens/total_calls

        
    
    
                

open_router_data = OpenRouterModelData.from_api_data(-1, only_free_models=True)

In [ ]:
import random


def generate_random_color():
    """Generate a random RGB color as a string"""
    return f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'

def create_figure_error_vs_price(router_model: OpenRouterModelData):
    # Normalize token sizes for bubble sizes
    max_tokens = max(router_model.model_total_tokens()) if router_model.model_total_tokens() else 1
    normalized_sizes = [(t / max_tokens) * 50 + 10 for t in router_model.model_total_tokens()]  # Scale for plotly

    # Figure 1: Interactive Error Rate vs Completion Price
    fig1 = go.Figure()

    

    unique_provider_color_dict = {provider:generate_random_color()  
                                  for provider in router_model.model_providers(unique_set=True)}

    # Create a separate trace for each provider (for legend)
    for provider in unique_provider_color_dict.keys():
        # Filter data for this provider
        provider_indices = [i for i, p in enumerate(router_model.model_providers()) if p == provider]
        
        fig1.add_trace(go.Scatter(
            x=[router_model.model_error_rates()[i] for i in provider_indices],
            y=[router_model.model_completion_prices()[i] for i in provider_indices],
            mode='markers',
            marker=dict(
                size=[normalized_sizes[i] for i in provider_indices],
                color=unique_provider_color_dict[provider],
                line=dict(color='white', width=2),
                opacity=0.7
            ),
            text=[router_model.model_model_names()[i] for i in provider_indices],
            customdata=np.column_stack((
                [router_model.model_total_tokens()[i] for i in provider_indices],
                [router_model.model_tool_calls()[i] for i in provider_indices],
                [router_model.model_providers()[i] for i in provider_indices]  # Added provider
            )),
            hovertemplate='<b>%{text}</b><br>' +
                        'Provider: <b>%{customdata[2]}</b><br>' + 
                        'Error Rate: %{x:.4f}<br>' +
                        'Price: $%{y}<br>' +
                        'Total Tokens: %{customdata[0]:,}<br>' +
                        'Tool Calls: %{customdata[1]:,}<br>' +
                        '<extra></extra>',
            name=provider  # This creates the legend entry
        ))

    fig1.update_layout(
        title='Model Performance: Error Rate vs Price Analysis<br><sub>Bubble size represents total token usage | Hover over dots to see details</sub>',
        xaxis_title='Tool Calling Error Rate',
        yaxis_title='Completion Price per M Token ($)',
        xaxis_type='log',
        yaxis_type='log',

        hovermode='closest',
        template='plotly_white',
        width=1400,
        height=900,
        font=dict(size=12),
        plot_bgcolor='#f8f9fa'
    )

    fig1.write_html('figure1_error_vs_price_interactive.html')
    print("✅ Created: figure1_error_vs_price_interactive.html")

create_figure_error_vs_price(OpenRouterModelData.from_api_data(1000, only_free_models=None))

In [ ]:
for model in open_router_data.model_data:
    if "meituan" in model.model_id:
        print(model)

In [ ]:
def create_figure_enhanced_analysis(router_model: OpenRouterModelData):
    # Figure 2: Enhanced visualization with log scale
    fig2 = go.Figure()

    # Normalize tool calls for color
    norm_tool_calls = np.array(router_model.model_tool_calls())
    norm_tool_calls = (norm_tool_calls - norm_tool_calls.min()) / (norm_tool_calls.max() - norm_tool_calls.min())
    
    # Normalize token sizes for bubble sizes
    max_tokens = max(router_model.model_total_tokens()) if router_model.model_total_tokens() else 1
    normalized_sizes = [(t / max_tokens) * 50 + 10 for t in router_model.model_total_tokens()]  # Scale for plotly
    
    # At the top of the function, after normalizing sizes:
    tool_calls_all = router_model.model_tool_calls()
    log_tool_calls = [np.log10(max(tc, 1)) for tc in tool_calls_all]  # log10, avoiding log(0)
    print("log_tool_calls = ", log_tool_calls)
    
    # Create a separate trace for each provider (for legend/filtering)
    for idx, provider in enumerate(sorted(router_model.model_providers(unique_set=True))):
        # Filter data for this provider
        provider_indices = [i for i, p in enumerate(router_model.model_providers()) if p == provider]
        
        fig2.add_trace(go.Scatter(
            x=[router_model.model_error_rates()[i] for i in provider_indices],
            y=[router_model.model_completion_prices()[i] for i in provider_indices],
            mode='markers',
            marker=dict(
                size=[normalized_sizes[i] for i in provider_indices],
                color=[log_tool_calls[i] for i in provider_indices],  # ← CHANGED: Use log values here!
                colorscale='RdYlBu_r',
                showscale=(idx == 0),  # Only show colorbar once (for first provider)
                colorbar=dict(
                    title="Tool Calls<br>(log scale)",  # Updated title
                    x=1.15,  # Move colorbar further right
                    xanchor='left',
                    len=0.7,  # Make it shorter
                    y=0.5,
                    yanchor='middle'
                ),
                line=dict(color='darkblue', width=1.5),
                opacity=0.8,
                cmin=min(log_tool_calls),  # Min of log values
                cmax=max(log_tool_calls)   # Max of log values
            ),
            text=[router_model.model_model_names()[i] for i in provider_indices],
            customdata=np.column_stack((
                [router_model.model_total_tokens()[i] for i in provider_indices],
                [router_model.model_tool_calls()[i] for i in provider_indices],
                [router_model.model_providers()[i] for i in provider_indices]
            )),
            hovertemplate='<b>%{text}</b><br>' +
                        'Provider: <b>%{customdata[2]}</b><br>' +
                        'Error Rate: %{x:.4f}<br>' +
                        'Price: $%{y:.6f}<br>' +
                        'Tool Calls: %{customdata[1]:,}<br>' +
                        'Total Tokens: %{customdata[0]:,}<br>' +
                        '<extra></extra>',
            name=provider,  # Provider name for legend (for filtering)
            legendgroup=provider  # Group by provider
        ))

    # Add reference lines
    if max(router_model.model_error_rates()) > 0:
        fig2.add_vline(x=0.01, line_dash="dash", line_color="red", opacity=0.5,
                    annotation_text="1% error threshold", annotation_position="top")
        fig2.add_vline(x=0.05, line_dash="dash", line_color="orange", opacity=0.5,
                    annotation_text="5% error threshold", annotation_position="top")

    fig2.update_layout(
        title='Model Efficiency Dashboard: Multi-dimensional Analysis<br><sub>Bubble size: Total tokens | Color: Tool call frequency (log scale) | Legend: Filter by provider</sub>',
        xaxis_title='Tool Calling Error Rate',
        yaxis_title='Completion Price per mil Token ($)',
        yaxis_type='log',
        xaxis_type='log',
        hovermode='closest',
        template='plotly_white',
        width=1400,
        height=900,
        font=dict(size=12),
        plot_bgcolor='#fafafa',
        showlegend=True,
        legend=dict(
            x=1.02,  # Position legend to the right
            y=1,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='black',
            borderwidth=1
        )
    )

    # Add stats annotation with corrected price formatting
    price_min = min(router_model.model_completion_prices())
    price_max = max(router_model.model_completion_prices())

    # Format prices based on their magnitude
    if price_max < 0.001:
        price_range_text = f"${price_min:.2e} - ${price_max:.2e}"
    elif price_max < 1:
        price_range_text = f"${price_min:.6f} - ${price_max:.6f}"
    else:
        price_range_text = f"${price_min:.4f} - ${price_max:.4f}"

    stats_text = (f"<b>Statistics:</b><br>"
                f"Models analyzed: {len(router_model.model_model_names())}<br>"
                f"Avg error rate: {np.mean(router_model.model_error_rates()):.4f}<br>"
                f"Price range: {price_range_text}<br>"
                f"Total tokens processed: {sum(router_model.model_total_tokens())/1e9:.2f}B")

    fig2.add_annotation(
        text=stats_text,
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        showarrow=False,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        xanchor='left',
        yanchor='top',
        align='left'
    )

    fig2.write_html('figure2_enhanced_analysis_interactive.html')
    print("✅ Created: figure2_enhanced_analysis_interactive.html")

create_figure_enhanced_analysis(OpenRouterModelData.from_api_data(100))

In [ ]:
def create_figure_performance_matrix(router_model: OpenRouterModelData):
    # Figure 3: Performance Matrix with subplots
    fig3 = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Error Rate by Model', 'Price vs Usage (Color: Error Rate)'),
        horizontal_spacing=0.12
    )

    # Left panel: Error rate distribution
    fig3.add_trace(
        go.Bar(
            y=router_model.model_model_names(),
            x=router_model.model_error_rates(),
            orientation='h',
            marker=dict(
                color=np.linspace(0, 1, len(router_model.model_error_rates())),
                colorscale='Viridis',
                line=dict(color='black', width=1)
            ),
            hovertemplate='<b>%{y}</b><br>Error Rate: %{x:.4f}<extra></extra>'
        ),
        row=1, col=1
    )

    # Right panel: Price vs Tokens scatter
    fig3.add_trace(
        go.Scatter(
            x=router_model.model_total_tokens(),
            y=router_model.model_completion_prices(),
            mode='markers',
            marker=dict(
                size=15,
                color=router_model.model_error_rates(),
                colorscale='RdYlGn_r',
                showscale=True,
                colorbar=dict(title="Error Rate", x=1.15),
                line=dict(color='black', width=1)
            ),
            text=router_model.model_model_names(),
            hovertemplate='<b>%{text}</b><br>' +
                        'Total Tokens: %{x:,}<br>' +
                        'Price: $%{y:.2e}<br>' +
                        '<extra></extra>'
        ),
        row=1, col=2
    )

    fig3.update_xaxes(title_text="Tool Calling Error Rate", row=1, col=1)
    fig3.update_xaxes(title_text="Total Tokens Used", type="log", row=1, col=2)
    fig3.update_yaxes(title_text="", row=1, col=1)
    fig3.update_yaxes(title_text="Completion Price per mil Token ($)", type="log", row=1, col=2)

    fig3.update_layout(
        title_text='Model Performance Analysis Suite<br><sub>Hover over elements to see details</sub>',
        showlegend=False,
        template='plotly_white',
        width=1600,
        height=800,
        font=dict(size=11),
        hovermode='closest'
    )

    fig3.write_html('figure3_performance_matrix_interactive.html')
    print("✅ Created: figure3_performance_matrix_interactive.html")

    print("\n💡 Open these HTML files in your browser to interact with them!")
    print("   Hover over any data point to see model details")


create_figure_performance_matrix(OpenRouterModelData.from_api_data(-1, only_free_models=True))

In [ ]:
import random


def generate_random_color():
    """Generate a random RGB color as a string"""
    return f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'

def create_figure_spend_vs_count(router_model: OpenRouterModelData):
    # Normalize error rates for bubble sizes (higher error = larger bubble to show problem areas)
    max_error = max(router_model.model_error_rates()) if router_model.model_error_rates() else 1
    normalized_sizes = [(e / max_error) * 50 + 10 for e in router_model.model_error_rates()]  # Scale for plotly

    # Figure: Interactive Total Spend vs Total Count
    fig1 = go.Figure()

    unique_provider_color_dict = {provider: generate_random_color()  
                                  for provider in router_model.model_providers(unique_set=True)}

    # Create a separate trace for each provider (for legend)
    for provider in unique_provider_color_dict.keys():
        # Filter data for this provider
        provider_indices = [i for i, p in enumerate(router_model.model_providers()) if p == provider]
        
        fig1.add_trace(go.Scatter(
            x=[router_model.total_count_used()[i] for i in provider_indices],
            y=[router_model.model_total_spend()[i] for i in provider_indices],
            mode='markers',
            marker=dict(
                size=[normalized_sizes[i] for i in provider_indices],
                color=unique_provider_color_dict[provider],
                line=dict(color='white', width=2),
                opacity=0.7
            ),
            text=[router_model.model_model_names()[i] for i in provider_indices],
            customdata=np.column_stack((
                [router_model.model_total_tokens()[i] for i in provider_indices],
                [router_model.model_tool_calls()[i] for i in provider_indices],
                [router_model.model_error_rates()[i] for i in provider_indices],
                [router_model.model_providers()[i] for i in provider_indices]  # Added provider
            )),
            hovertemplate='<b>%{text}</b><br>' +
                        'Provider: <b>%{customdata[3]}</b><br>' + 
                        'Total Count: %{x:,}<br>' +
                        'Total Spend: $%{y:.4f}<br>' +
                        'Total Tokens: %{customdata[0]:,}<br>' +
                        'Tool Calls: %{customdata[1]:,}<br>' +
                        'Error Rate: %{customdata[2]:.4f}<br>' +
                        '<extra></extra>',
            name=provider  # This creates the legend entry
        ))

    fig1.update_layout(
        title='Model Usage: Total Spend vs Total Count<br><sub>Bubble size represents error rate | Color represents provider</sub>',
        xaxis_title='Total Count (Number of Requests)',
        yaxis_title='Total Spend ($)',
        xaxis_type='log',
        yaxis_type='log',
        hovermode='closest',
        template='plotly_white',
        width=1400,
        height=900,
        font=dict(size=12),
        plot_bgcolor='#f8f9fa',
        showlegend=True
    )

    fig1.write_html('figure4_spend_vs_count_interactive.html')
    print("✅ Created: figure4_spend_vs_count_interactive.html")

create_figure_spend_vs_count(OpenRouterModelData.from_api_data(1000, only_free_models=None))

In [ ]:
def create_figure_spend_vs_price(router_model: OpenRouterModelData):
    # Normalize error rates for bubble sizes (higher error = larger bubble to show problem areas)
    max_error = max(router_model.model_error_rates()) if router_model.model_error_rates() else 1
    normalized_sizes = [(e / max_error) * 50 + 10 for e in router_model.model_error_rates()]  # Scale for plotly

    # Figure: Interactive Total Spend vs Total Count
    fig1 = go.Figure()

    unique_provider_color_dict = {provider: generate_random_color()  
                                  for provider in router_model.model_providers(unique_set=True)}

    # Create a separate trace for each provider (for legend)
    for provider in unique_provider_color_dict.keys():
        # Filter data for this provider
        provider_indices = [i for i, p in enumerate(router_model.model_providers()) if p == provider]
        
        fig1.add_trace(go.Scatter(
            x=[router_model.total_count_used()[i] for i in provider_indices],
            y=[router_model.model_total_spend()[i] for i in provider_indices],
            mode='markers',
            marker=dict(
                size=[normalized_sizes[i] for i in provider_indices],
                color=unique_provider_color_dict[provider],
                line=dict(color='white', width=2),
                opacity=0.7
            ),
            text=[router_model.model_model_names()[i] for i in provider_indices],
            customdata=np.column_stack((
                [router_model.model_total_tokens()[i] for i in provider_indices],
                [router_model.model_tool_calls()[i] for i in provider_indices],
                [router_model.model_error_rates()[i] for i in provider_indices],
                [router_model.model_providers()[i] for i in provider_indices]  # Added provider
            )),
            hovertemplate='<b>%{text}</b><br>' +
                        'Provider: <b>%{customdata[3]}</b><br>' + 
                        'Total Count: %{x:,}<br>' +
                        'Total Spend: $%{y:.4f}<br>' +
                        'Total Tokens: %{customdata[0]:,}<br>' +
                        'Tool Calls: %{customdata[1]:,}<br>' +
                        'Error Rate: %{customdata[2]:.4f}<br>' +
                        '<extra></extra>',
            name=provider  # This creates the legend entry
        ))

    fig1.update_layout(
        title='Model Usage: Total Spend vs Total Count<br><sub>Bubble size represents error rate | Color represents provider</sub>',
        xaxis_title='Total Count (Number of Requests)',
        yaxis_title='Total Spend ($)',
        xaxis_type='log',
        yaxis_type='log',
        hovermode='closest',
        template='plotly_white',
        width=1400,
        height=900,
        font=dict(size=12),
        plot_bgcolor='#f8f9fa',
        showlegend=True
    )

    fig1.write_html('figure4_spend_vs_count_interactive.html')
    print("✅ Created: figure4_spend_vs_count_interactive.html")

create_figure_spend_vs_count(OpenRouterModelData.from_api_data(1000, only_free_models=None))

In [ ]:
def create_figure_spend_vs_price(router_model: OpenRouterModelData):
    # Normalize error rates for bubble sizes (higher error = larger bubble to show problem areas)
    max_error = max(router_model.model_error_rates()) if router_model.model_error_rates() else 1
    normalized_sizes = [(e / max_error) * 50 + 10 for e in router_model.model_error_rates()]  # Scale for plotly

    # Figure: Interactive Total Spend vs Completion Price
    fig1 = go.Figure()

    unique_provider_color_dict = {provider: generate_random_color()  
                                  for provider in router_model.model_providers(unique_set=True)}

    # Create a separate trace for each provider (for legend)
    for provider in sorted(unique_provider_color_dict.keys()):
        # Filter data for this provider
        provider_indices = [i for i, p in enumerate(router_model.model_providers()) if p == provider]
        
        fig1.add_trace(go.Scatter(
            x=[router_model.model_completion_prices()[i] for i in provider_indices],
            y=[router_model.model_total_spend()[i] for i in provider_indices],
            mode='markers',
            marker=dict(
                size=[normalized_sizes[i] for i in provider_indices],
                color=unique_provider_color_dict[provider],
                line=dict(color='white', width=2),
                opacity=0.7
            ),
            text=[router_model.model_model_names()[i] for i in provider_indices],
            customdata=np.column_stack((
                [router_model.model_total_tokens()[i] for i in provider_indices],
                [router_model.model_tool_calls()[i] for i in provider_indices],
                [router_model.model_error_rates()[i] for i in provider_indices],
                [router_model.total_count_used()[i] for i in provider_indices],
                [router_model.model_providers()[i] for i in provider_indices]  # Added provider
            )),
            hovertemplate='<b>%{text}</b><br>' +
                        'Provider: <b>%{customdata[4]}</b><br>' + 
                        'Completion Price: $%{x:,}<br>' +
                        'Total Spend: $%{y:.4f}<br>' +
                        'Total Count: %{customdata[3]:,}<br>' +
                        'Total Tokens: %{customdata[0]:,}<br>' +
                        'Tool Calls: %{customdata[1]:,}<br>' +
                        'Error Rate: %{customdata[2]:.4f}<br>' +
                        '<extra></extra>',
            name=provider  # This creates the legend entry
        ))

    fig1.update_layout(
        title='Model Usage: Total Spend vs Completion Price<br><sub>Bubble size represents error rate | Color represents provider</sub>',
        xaxis_title='Completion price per M token ($)',
        yaxis_title='Total Spend ($)',
        xaxis_type='log',
        yaxis_type='log',
        hovermode='closest',
        template='plotly_white',
        width=1400,
        height=900,
        font=dict(size=12),
        plot_bgcolor='#f8f9fa',
        showlegend=True
    )

    fig1.write_html('figure5_spend_vs_price_interactive.html')
    print("✅ Created: figure5_spend_vs_price_interactive.html")

create_figure_spend_vs_price(OpenRouterModelData.from_api_data(-1, only_free_models=None))

In [ ]:
def create_figure_error_rate_vs_usage(router_model: OpenRouterModelData):
    # Normalize total spend for bubble sizes (higher spend = larger bubble)
    max_spend = max(router_model.total_count_used()) if router_model.total_count_used() else 1
    normalized_sizes = [(s / max_spend) * 50 + 10 for s in router_model.total_count_used()]  # Scale for plotly

    # Figure 7: Interactive Error Rate vs Usage (Total Count)
    fig7 = go.Figure()

    unique_provider_color_dict = {provider: generate_random_color()  
                                  for provider in router_model.model_providers(unique_set=True)}

    # Create a separate trace for each provider (for legend)
    for provider in unique_provider_color_dict.keys():
        # Filter data for this provider
        provider_indices = [i for i, p in enumerate(router_model.model_providers()) if p == provider]
        
        fig7.add_trace(go.Scatter(
            x=[router_model.total_count_used()[i] for i in provider_indices],
            y=[router_model.model_error_rates()[i] for i in provider_indices],
            mode='markers',
            marker=dict(
                size=[normalized_sizes[i] for i in provider_indices],
                color=unique_provider_color_dict[provider],
                line=dict(color='white', width=2),
                opacity=0.7
            ),
            text=[router_model.model_model_names()[i] for i in provider_indices],
            customdata=np.column_stack((
                [router_model.model_total_tokens()[i] for i in provider_indices],
                [router_model.model_tool_calls()[i] for i in provider_indices],
                [router_model.total_count_used()[i] for i in provider_indices],
                [router_model.model_providers()[i] for i in provider_indices]  # Added provider
            )),
            hovertemplate='<b>%{text}</b><br>' +
                        'Provider: <b>%{customdata[3]}</b><br>' + 
                        'Total Count: %{x:,}<br>' +
                        'Error Rate: %{y:.4f}<br>' +
                        'Total Tokens: %{customdata[0]:,}<br>' +
                        'Tool Calls: %{customdata[1]:,}<br>' +
                        'Total Calls: %{customdata[2]:,}<br>' +
                        '<extra></extra>',
            name=provider  # This creates the legend entry
        ))

    # Add reference lines for error rate thresholds
    max_count = max(router_model.total_count_used()) if router_model.total_count_used() else 1
    
    fig7.add_hline(y=0.01, line_dash="dash", line_color="red", opacity=0.5,
                   annotation_text="1% error threshold", annotation_position="right")
    fig7.add_hline(y=0.05, line_dash="dash", line_color="orange", opacity=0.5,
                   annotation_text="5% error threshold", annotation_position="right")

    fig7.update_layout(
        title='Free Models: Model Reliability: Error Rate vs Usage<br><sub>Bubble size represents total spend | Color represents provider | Hover for details</sub>',
        xaxis_title='Total Count (Number of Requests)',
        yaxis_title='Error Rate',
        xaxis_type='log',
        hovermode='closest',
        template='plotly_white',
        width=1400,
        height=900,
        font=dict(size=12),
        plot_bgcolor='#f8f9fa',
        showlegend=True,
        legend=dict(
            title=dict(text='Provider'),
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    # Add stats annotation
    avg_error = np.mean(router_model.model_error_rates())
    total_requests = sum(router_model.total_count_used())
    
    stats_text = (f"<b>Statistics:</b><br>"
                  f"Models analyzed: {len(router_model.model_model_names())}<br>"
                  f"Avg error rate: {avg_error:.4f}<br>"
                  f"Total requests: {total_requests:,}<br>"
                  f"Total calls: ${sum(router_model.total_count_used()):.2f}")

    fig7.add_annotation(
        text=stats_text,
        xref="paper", yref="paper",
        x=0.98, y=0.02,
        showarrow=False,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        xanchor='right',
        yanchor='bottom',
        align='left'
    )

    fig7.write_html('figure7_error_rate_vs_usage_free_models.html')
    print("✅ Created: figure7_error_rate_vs_usage_free_models.html")

create_figure_error_rate_vs_usage(OpenRouterModelData.from_api_data(1000, only_free_models=True))

In [ ]:
def create_figure_avg_tokens_vs_price(router_model: OpenRouterModelData):
    # Normalize error rates for bubble sizes (higher error = larger bubble to show problem areas)
    max_error = max(router_model.model_error_rates()) if router_model.model_error_rates() else 1
    normalized_sizes = [(e / max_error) * 50 + 10 for e in router_model.model_error_rates()]  # Scale for plotly

    # Figure: Interactive Average Tokens vs Completion Price
    fig8 = go.Figure()

    unique_provider_color_dict = {provider: generate_random_color()  
                                  for provider in router_model.model_providers(unique_set=True)}

    # Create a separate trace for each provider (for legend)
    for provider in sorted(unique_provider_color_dict.keys()):
        # Filter data for this provider
        provider_indices = [i for i, p in enumerate(router_model.model_providers()) if p == provider]
        
        fig8.add_trace(go.Scatter(
            x=[router_model.model_completion_prices()[i] for i in provider_indices],
            y=[router_model.tokens_per_request()[i] for i in provider_indices],
            mode='markers',
            marker=dict(
                size=[normalized_sizes[i] for i in provider_indices],
                color=unique_provider_color_dict[provider],
                line=dict(color='white', width=2),
                opacity=0.7
            ),
            text=[router_model.model_model_names()[i] for i in provider_indices],
            customdata=np.column_stack((
                [router_model.model_total_tokens()[i] for i in provider_indices],
                [router_model.model_tool_calls()[i] for i in provider_indices],
                [router_model.model_error_rates()[i] for i in provider_indices],
                [router_model.total_count_used()[i] for i in provider_indices],
                [router_model.model_providers()[i] for i in provider_indices]  # Added provider
            )),
            hovertemplate='<b>%{text}</b><br>' +
                        'Provider: <b>%{customdata[4]}</b><br>' + 
                        'Completion Price: $%{x:,}<br>' +
                        'Avg Tokens per Request: %{y:,.0f}<br>' +
                        'Total Count: %{customdata[3]:,}<br>' +
                        'Total Tokens: %{customdata[0]:,}<br>' +
                        'Tool Calls: %{customdata[1]:,}<br>' +
                        'Error Rate: %{customdata[2]:.4f}<br>' +
                        '<extra></extra>',
            name=provider  # This creates the legend entry
        ))

    fig8.update_layout(
        title='Model Usage: Average Tokens per Request vs Completion Price<br><sub>Bubble size represents error rate | Color represents provider</sub>',
        xaxis_title='Completion price per M token ($)',
        yaxis_title='Average Tokens per Request',
        xaxis_type='log',
        yaxis_type='log',
        hovermode='closest',
        template='plotly_white',
        width=1400,
        height=900,
        font=dict(size=12),
        plot_bgcolor='#f8f9fa',
        showlegend=True
    )

    fig8.write_html('figure8_avg_tokens_vs_price_interactive.html')
    print("✅ Created: figure8_avg_tokens_vs_price_interactive.html")

create_figure_avg_tokens_vs_price(OpenRouterModelData.from_api_data(-1, only_free_models=None))